# Recommandation avec cold start

In [1]:
import pandas as pd

import seaborn as sns

from deep_implicit_feedback_recsys import *

from cold_star_recommender import *

from recommender_evaluation import *

Using TensorFlow backend.


In [2]:
def normalized_results(df, recommender_type):
    df['average_reward_normalized_max'] = df['average_reward'] / df['max_price']
    df['average_reward_normalized_mean'] = df['average_reward'] / df['mean_price']
    df['recommendation'] = recommender_type
    return df

In [3]:
n_loop = 10
n_pred = 1000
online_batch_size = n_pred // 10

user_dim = item_dim = 8
n_hidden = 1
hidden_size = 64
dropout = 0.1
l2_reg = 0

ID = 'MC15CHD1JU24INEKKPU4'


## Mesures d'échantillon

### Implicit Recommender avec nuls sans indicateur, en ligne


In [ ]:
%%time
ImplicitRecommenderWithNull_no_indicator_online_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull_no_indicator,
                                           n_loop=n_loop, n_pred=n_pred,
                                           online_batch_size=1,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg)


In [ ]:
ImplicitRecommenderWithNull_no_indicator_online_df_normalized = normalized_results(ImplicitRecommenderWithNull_no_indicator_online_df,
                                                                                   'Null, en ligne')
ImplicitRecommenderWithNull_no_indicator_online_df_normalized

### Implicit Recommender avec nuls et indicateur special, hors ligne

In [ ]:
%%time
ImplicitRecommenderWithNull_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull,
                                           n_loop=n_loop, n_pred=n_pred,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg)
                                           

In [ ]:
ImplicitRecommenderWithNull_df_normalized = normalized_results(ImplicitRecommenderWithNull_df, 'Null special, hors ligne')
ImplicitRecommenderWithNull_df_normalized

### Implicit Recommender avec nuls et indicateur special, en ligne


In [ ]:
%%time
ImplicitRecommenderWithNull_online_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull,
                                           n_loop=n_loop, n_pred=n_pred,
                                           online_batch_size=1,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg)


In [ ]:
ImplicitRecommenderWithNull_online_df_normalized = normalized_results(ImplicitRecommenderWithNull_online_df, 'Null special, en ligne')
ImplicitRecommenderWithNull_online_df_normalized

### Implicit Recommender avec nuls et indicateur special, en ligne version batch


In [ ]:
%%time
ImplicitRecommenderWithNull_online_batch_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull,
                                           n_loop=n_loop, n_pred=n_pred,
                                           online_batch_size=online_batch_size,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg)


In [ ]:
ImplicitRecommenderWithNull_online_batch_df_normalized = \
    normalized_results(ImplicitRecommenderWithNull_online_batch_df, 'Null special, en ligne (batch)')
ImplicitRecommenderWithNull_online_batch_df_normalized

### Fully Implicit Recommender avec nuls et indicateur binaire, en ligne version batch


In [ ]:
%%time
FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartFullyImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull_binary_indicator,
                                           n_loop=n_loop, n_pred=n_pred,
                                           online_batch_size=online_batch_size,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg)


In [ ]:
FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df_normalized = \
    normalized_results(FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df, 'Fully Implicit Null binaire, en ligne (batch)')
FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df_normalized

## Aggregation des résutats

In [ ]:
all_results = pd.concat( (
    ImplicitRecommenderWithNull_no_indicator_online_df_normalized,
    ImplicitRecommenderWithNull_df_normalized,
    ImplicitRecommenderWithNull_online_df_normalized,
    ImplicitRecommenderWithNull_online_batch_df_normalized,
    
    FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df_normalized,
), axis = 0)

In [ ]:
aggregat = all_results.groupby('recommendation').mean()

### Classement ratio de recommandations effectives

In [ ]:
col = 'good_reco_ratio' 
aggregat.sort_values(col, ascending=False)\
    .style.apply((lambda x: ['background: lightgreen' if x.name == col else '' for _ in x]))

### Classement par average reward normalisée (max)

In [ ]:
col = 'average_reward_normalized_max' 
aggregat.sort_values(col, ascending=False)\
    .style.apply((lambda x: ['background: lightgreen' if x.name == col else '' for _ in x]))

### Classement par average reward normalisée (mean)

In [ ]:
col = 'average_reward_normalized_mean' 
aggregat.sort_values(col, ascending=False)\
    .style.apply((lambda x: ['background: lightgreen' if x.name == col else '' for _ in x]))

### Résumé (classé par average_reward)

In [ ]:
cm = sns.light_palette("green", as_cmap=True)
col = 'average_reward' 
aggregat.sort_values(col, ascending=False)\
    .style.background_gradient(cmap=cm).highlight_max(color='red')

La recommendation prenant en charge les nulls semble meilleure. <br>
  1) Null secial en ligne (batch) <br>
  2) Fully Implicit Null special, en ligne (batch) <br>
  3) Null, hors ligne <br>